In [2]:
!pip install selenium
!pip install webdriver-manager

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import requests

In [4]:
def scroll_to_bottom(driver):
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    for i in range(0, total_height, 10):
        driver.execute_script("window.scrollTo(0, {});".format(i))

In [5]:
def scrape_istock_images(animal_name='harimau', total_images=700):
    # Set up the webdriver
    options = Options()
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        if not os.path.exists(animal_name):
            os.makedirs(animal_name)
        
        animal = animal_name.replace(" ", "%20").replace("-", "%20")
        
        base_url = f"https://www.istockphoto.com/id/search/2/image?phrase=harimau&mediatype=photography"
        current_page = 1
        downloaded_images = 0

        while downloaded_images < total_images:
            url = f"{base_url}{current_page}"
            driver.get(url)

            scroll_to_bottom(driver)

            image_elements = driver.find_elements(By.XPATH, '//article/a/figure/picture/img')

            if not image_elements:
                print("No images found on the current page.")
                break

            for img in image_elements:
                if downloaded_images < total_images:
                    img_url = img.get_attribute('src')
                    img_name = f"{animal_name}_{downloaded_images + 1}.jpg"
                    img_path = os.path.join(animal_name, img_name)

                    try:
                        img_data = requests.get(img_url).content
                        with open(img_path, 'wb') as file:
                            file.write(img_data)
                            print(f"Downloaded: {img_name}")
                            downloaded_images += 1
                    except Exception as e:
                        print(f"Failed to download {img_name}: {str(e)}")
                else:
                    break

            print(f"Images on page {current_page} are downloaded, opening next page...")

            current_page += 1

    finally:
        driver.quit()

In [6]:
# animals = [
#     animal name, base photo id
#     ['turtle-tortoise', '2168961245'],
#     ['seahorse', '2066522237'],
#     ['sea urchins', '1333399400'],
#     ['nudibranchs', '488915173'],
#     ['octopus', '1136409408'],
#     ['pufferfish', '163014234'],
#     ['rays', '540756336'],
#     ['whales', '1164479946'],
#     ['crabs', '177207059'],
#     ['squids', '1176905301'],
#     ['dolphins', '903018736'],
#     ['starfish', '1002745574'],
#     ['lobsters', '1366100593'],
#     ['jellyfish', '863572758'],
#     ['sharks', '1438305496'],
#     ['corals', '1077171172'],
#     ['manatees', '1393927299']
# ]

In [7]:
# for animal in animals:
#     scrape_istock_images(animal[1], animal[0], total_images=1000)

In [8]:
scrape_istock_images(total_images=10)

WebDriverException: Message: Service /home/zwniff/.wdm/drivers/chromedriver/linux64/114.0.5735.90/chromedriver unexpectedly exited. Status code was: 127
